In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk...."

In [ ]:
!pip install -q youtube-transcript-api


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 9.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 7.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#Imports

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# loading data(collect all the books shelves)

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi


video_id = "4gEhj3CqGSw" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    ytt_api = YouTubeTranscriptApi()
    transcript_list=ytt_api.fetch(video_id)

    # Flatten it to plain text
    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

In India's digital capital, UPI was once king. But now, Bangalore vendors are tearing down QR codes and saying cash only. Why are the same street sellers who embrace digital now running from it? Because behind every beep of a UPI payment, there could be a tax notice. Hi, this is Katija and you're watching the free press journal. Today we are decoding a crisis that's turning India's digital dream upside down. Why are Bangaluru's food stalls, corner shops and small vendors refusing UPI payments? Why are posters saying no UPI only cash spreading in the city that once led India's fine tech revolution? In this video, we'll explain how GST enforcement triggered panic, what UPI has to do with tax notices, how the Karnataka government is trying to calm things down, and why traders are now planning a strike. Let's get into the full story. Imagine you are at a tea stall in Kora Mangala. You take out your phone to pay via UPI and the seller says cash only. You look around and you see the same thi

# text splitter(split book shelves in books)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=30)
#texts = text_splitter.split_text(transcript)
chunks = text_splitter.create_documents([transcript])

In [ ]:
chunks

[Document(metadata={}, page_content="In India's digital capital, UPI was once king. But now, Bangalore vendors are tearing down QR codes and saying cash only. Why are the same street"),
 Document(metadata={}, page_content='only. Why are the same street sellers who embrace digital now running from it? Because behind every beep of a UPI payment, there could be a tax'),
 Document(metadata={}, page_content="payment, there could be a tax notice. Hi, this is Katija and you're watching the free press journal. Today we are decoding a crisis that's turning"),
 Document(metadata={}, page_content="a crisis that's turning India's digital dream upside down. Why are Bangaluru's food stalls, corner shops and small vendors refusing UPI payments? Why"),
 Document(metadata={}, page_content="refusing UPI payments? Why are posters saying no UPI only cash spreading in the city that once led India's fine tech revolution? In this video, we'll"),
 Document(metadata={}, page_content="In this video, we'll expla

# embedding(convert every book into embedding to get sematic idea)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

# hard way(look for every book in library)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

v1 = embeddings_model.embed_query("What is UPI?")
v2 = embeddings_model.embed_query("Tell me about digital payments.")

similarity = cosine_similarity([v1], [v2])
print(len(v1))
print(similarity)

3072
[[0.37075167]]


# create smart library

In [ ]:
from langchain.vectorstores import FAISS

# chunks should be a list of langchain Documents
vector_store = FAISS.from_documents(chunks, embeddings_model)

# find most similar books with scores(semantically similar)(low score = more similar)

In [ ]:
query = "areas in city"
similar_chunks = vector_store.similarity_search_with_score(query, k=5)

for i, (doc, score) in enumerate(similar_chunks, 1):
    print(f"Match {i} (Score: {score:.4f}): {doc.page_content}")

Match 1 (Score: 1.5085): you are at a tea stall in Kora Mangala. You take out your phone to pay via UPI and the seller says cash only. You look around and you see the same
Match 2 (Score: 1.5595): schemes and Congress MLAs are demanding more funds for roads, hospitals, housing and urban projects. To fill the gap, the tax department has been
Match 3 (Score: 1.5644): In India's digital capital, UPI was once king. But now, Bangalore vendors are tearing down QR codes and saying cash only. Why are the same street
Match 4 (Score: 1.5843): they are tiny food stalls or roadside vendors. And the easiest way to find them, UPI data, clean, traceable, and digital. But here's where the real
Match 5 (Score: 1.5867): refusing UPI payments? Why are posters saying no UPI only cash spreading in the city that once led India's fine tech revolution? In this video, we'll


In [ ]:
vector_store.save_local("my_faiss_index")

In [ ]:
vector_store = FAISS.load_local(
    "my_faiss_index",
    embeddings_model,
    allow_dangerous_deserialization=True
)

# Smart librarian(Takes your query searches through the library and brings you most similar books)

In [ ]:
retriever = vector_store.as_retriever()
results = retriever.invoke("What is water?")
print(results)

[Document(id='378eb08c-5c90-44f3-ac7b-882bf3156faa', metadata={}, page_content="they are tiny food stalls or roadside vendors. And the easiest way to find them, UPI data, clean, traceable, and digital. But here's where the real"), Document(id='11a3de83-1c5c-48bd-a29d-04ab67a7db61', metadata={}, page_content="tax liability? Now, very importantly, why is this happening now? Now, here's the question everyone's asking. Why now? Why are small traders being"), Document(id='07384074-9832-4a50-b006-fdc11c57840c', metadata={}, page_content="Why? Firstly, because cash doesn't leave a trail. Second, no records, no transaction history. Third, no data to be used against them later. So"), Document(id='bbffff2a-3302-4106-b3d6-393f85f3bf74', metadata={}, page_content="goods like milk. The message is very clear. They won't back down unless the government listens. In response, Chief Minister Sida Ramaya, who also")]


#we can tell librarian, use mmr or similarity or similarity_score_threshold thats why we need librarian

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",     # or "similarity"
    search_kwargs={"k": 5}
)

docs = retriever.invoke("What is water?")
print(docs)

[Document(id='ee3d429f-574d-46ec-a531-c4a1a838b253', metadata={}, page_content="they are tiny food stalls or roadside vendors. And the easiest way to find them, UPI data, clean, traceable, and digital. But here's where the real"), Document(id='938e4a78-f9d2-40b1-9500-6652dfdb5d3f', metadata={}, page_content="to the no GST campaign. But many says it's too little too late. So lastly, where do we go from here? Let's take a step back. Everyone agrees tax"), Document(id='ffb529da-98df-4bd3-8439-00fb696ccd4c', metadata={}, page_content="Why? Firstly, because cash doesn't leave a trail. Second, no records, no transaction history. Third, no data to be used against them later. So"), Document(id='0f5cf308-a4df-40f8-99ce-73325d3ed962', metadata={}, page_content='Second a statewide shop on July 25. Third, and if no action is taken, they have even threatened to stop selling essential goods like milk. The'), Document(id='fc995bcf-85e0-4b46-b39f-954c2cd76a8c', metadata={}, page_content="no proof, no 

# Augmentation
##(You feed books librarian bought to chatgpt and query it, chatgpt understands the books, and query and gives better answer based on its knowledge)
##“Hey ChatGPT, here's what I found in my books 📚 — now please help me answer the question.”

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = "is the topic of gst discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)
print(retrieved_docs)

[Document(id='8fcb0b18-6540-478f-8591-6ede7bac4095', metadata={}, page_content="In this video, we'll explain how GST enforcement triggered panic, what UPI has to do with tax notices, how the Karnataka government is trying to calm"), Document(id='1398413d-60d1-4734-b612-72c11b0025f3', metadata={}, page_content="next GST council meeting, and he has asked tax officials to talk to traders directly to resolve this. That's what led to the no GST campaign. But"), Document(id='bb19ca8f-105b-4824-83d3-94f0d73af7cf', metadata={}, page_content="to the no GST campaign. But many says it's too little too late. So lastly, where do we go from here? Let's take a step back. Everyone agrees tax"), Document(id='80e9a81a-85f0-4d9d-8939-8dbd1368889d', metadata={}, page_content='Sida Ramaya, who also handles the finance department, has stepped in. He promised that the issue will be raised at the next GST council meeting, and')]


In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"In this video, we'll explain how GST enforcement triggered panic, what UPI has to do with tax notices, how the Karnataka government is trying to calm\n\nnext GST council meeting, and he has asked tax officials to talk to traders directly to resolve this. That's what led to the no GST campaign. But\n\nto the no GST campaign. But many says it's too little too late. So lastly, where do we go from here? Let's take a step back. Everyone agrees tax\n\nSida Ramaya, who also handles the finance department, has stepped in. He promised that the issue will be raised at the next GST council meeting, and"

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of GST is discussed in this video. It explains how GST enforcement triggered panic among traders, the burden of proof under GST law, and the concerns regarding tax notices related to digital payments. It also mentions the Karnataka government's efforts to address these issues and the promise to raise the matter at the next GST council meeting.


# Runnable

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain.vectorstores import FAISS

# chunks should be a list of langchain Documents
vector_store = FAISS.from_documents(chunks, embeddings_model)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain_core.runnables import RunnableMap
from langchain_core.documents import Document

# Step to get docs from retriever and join as a string
def get_context(question: str):
    docs = retriever.invoke(question)
    return "\n\n".join(doc.page_content for doc in docs)

parallel_chain = RunnableMap({
    "question": RunnablePassthrough(),       # keep original question
    "context": get_context         # fetch & format docs
})

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, just say you don't know.

{context}
Question: {question}
"""
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('is the topic related to gst discussed? give me summary in 5 points')

'Yes, the topic related to GST is discussed. Here is a summary in 5 points:\n\n1. The video explains how GST enforcement has caused panic among traders, leading to a campaign against GST.\n2. There is a connection between UPI (Unified Payments Interface) and tax notices, which is affecting traders.\n3. The Karnataka government is attempting to address the concerns raised by the no GST campaign, but many believe their efforts are insufficient.\n4. The discussion highlights the implications of GST law, particularly regarding the tax trap for traders with an annual turnover exceeding 40 lakh rupees.\n5. The next GST council meeting is anticipated, with a focus on direct communication between tax officials and traders to resolve ongoing issues.'